In [1]:
import scipy.io
import numpy as np
import CMR_IA as cmr
import matplotlib.pyplot as plt
import pickle
from statistics import mean, stdev
import pprint
import import_ipynb
import time
import pandas as pd
import seaborn as sns

np.set_printoptions(edgeitems=20, linewidth=150,precision=4)

In [2]:
# get wordfreq quantile
with open('../Data/simu4_word_freq.pkl', 'rb') as inp:
    df = pickle.load(inp)

In [3]:
with open("../Data/simu4_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)

In [4]:
s_mat = np.load('../Data/simu4_smat.npy')

In [5]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.4,
    beta_rec = 0.4,
    beta_rec_post = 0.2,
    gamma_fc = 0.5,
    s_fc = 0,
    c_thresh = 0.18,
    c_thresh_ass = 0.18,
    a = 2800,
    b = 2.14,
    m = 0,
    n = 1,
)
params

{'beta_enc': 0.4,
 'beta_rec': 0.4,
 'beta_rec_new': 0.5,
 'beta_rec_post': 0.2,
 'beta_distract': None,
 'phi_s': 2,
 'phi_d': 0.5,
 's_cf': 0,
 's_fc': 0,
 'kappa': 0.5,
 'eta': 0.5,
 'omega': 8,
 'alpha': 4,
 'c_thresh': 0.18,
 'c_thresh_ass': 0.18,
 'lamb': 0.5,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': False,
 'a': 2800,
 'b': 2.14,
 'm': 0,
 'n': 1,
 'gamma_fc': 0.5,
 'gamma_cf': 0.5}

In [6]:
df_study = df_study.iloc[0:6]
df_study.columns = df_study.columns.str.replace('itemno', 'itemno1')
df_study['itemno1'] = [1,2,3,4,5,11]
df_study['itemno2'] = [6,7,8,9,10,12]
df_study = df_study[['itemno1','itemno2', 'session']]
df_study

,itemno1,itemno2,session
0,1,6,0
1,2,7,0
2,3,8,0
3,4,9,0
4,5,10,0
5,11,12,0


In [7]:
df_test = df_study.copy()
df_test['itemno1'] = [1,10,3,11,4,11]
df_test['itemno2'] = [6,-1,7,-1,9,12]
df_test

,itemno1,itemno2,session
0,1,6,0
1,10,-1,0
2,3,7,0
3,11,-1,0
4,4,9,0
5,11,12,0


In [8]:
df_thin = df_test[['session','itemno1', 'itemno2']]
df_thin = df_thin.assign(s_resp=np.nan, s_rt=np.nan, csim=np.nan)
df_thin

,session,itemno1,itemno2,s_resp,s_rt,csim
0,0,1,6,NaN,NaN,NaN
1,0,10,-1,NaN,NaN,NaN
2,0,3,7,NaN,NaN,NaN
3,0,11,-1,NaN,NaN,NaN
4,0,4,9,NaN,NaN,NaN
5,0,11,12,NaN,NaN,NaN


In [9]:
# extarct the session data
sess = 0
pres_mat = df_study.loc[df_study.session==sess,['itemno1','itemno2']].to_numpy()
pres_mat = np.reshape(pres_mat,(len(pres_mat),1,2))
pres_mat

array([[[ 1,  6]],

       [[ 2,  7]],

       [[ 3,  8]],

       [[ 4,  9]],

       [[ 5, 10]],

       [[11, 12]]])

In [10]:
cue_mat = df_thin.loc[df_thin.session==sess,['itemno1','itemno2']].to_numpy()
cue_mat

array([[ 1,  6],
       [10, -1],
       [ 3,  7],
       [11, -1],
       [ 4,  9],
       [11, 12]])

In [11]:
sem_mat = s_mat

In [12]:
model = cmr.CMR2(params, pres_mat, sem_mat, source_mat=None,
           rec_mat=None, ffr_mat=None, cue_mat=cue_mat, task='Recog', mode='Hockley')

In [13]:
model.pres_nos

array([[[ 1,  6]],

       [[ 2,  7]],

       [[ 3,  8]],

       [[ 4,  9]],

       [[ 5, 10]],

       [[11, 12]]], dtype=int16)

In [14]:
model.cues_nos

array([[ 1,  6],
       [10, -1],
       [ 3,  7],
       [11, -1],
       [ 4,  9],
       [11, 12]], dtype=int16)

In [15]:
model.all_nos_unique

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int16)

In [16]:
model.pres_indexes

array([[[ 0,  5]],

       [[ 1,  6]],

       [[ 2,  7]],

       [[ 3,  8]],

       [[ 4,  9]],

       [[10, 11]]])

In [17]:
model.cues_indexes

array([[ 0,  5],
       [ 9, -1],
       [ 2,  6],
       [10, -1],
       [ 3,  8],
       [10, 11]])

In [18]:
model.sem_mat

array([[ 0.    ,  0.0438,  0.0616,  0.2806,  0.2136,  0.2108,  0.0681,  0.0047,  0.1484,  0.3749,  0.1175,  0.2497],
       [ 0.0438,  0.    ,  0.1883,  0.0561,  0.1495,  0.0434,  0.0774,  0.0962,  0.1081,  0.0624,  0.2443,  0.0995],
       [ 0.0616,  0.1883,  0.    ,  0.1312,  0.3007,  0.1605,  0.0623,  0.1676,  0.3182,  0.0529,  0.2136,  0.159 ],
       [ 0.2806,  0.0561,  0.1312,  0.    ,  0.2054,  0.1873,  0.0568,  0.1696,  0.1597,  0.1162,  0.0794,  0.44  ],
       [ 0.2136,  0.1495,  0.3007,  0.2054,  0.    ,  0.1337,  0.1497,  0.1188,  0.5148,  0.1597,  0.386 ,  0.2018],
       [ 0.2108,  0.0434,  0.1605,  0.1873,  0.1337,  0.    ,  0.0444,  0.2043,  0.1778,  0.1262,  0.0961,  0.2452],
       [ 0.0681,  0.0774,  0.0623,  0.0568,  0.1497,  0.0444,  0.    ,  0.0648, -0.0474,  0.0846, -0.0304,  0.0566],
       [ 0.0047,  0.0962,  0.1676,  0.1696,  0.1188,  0.2043,  0.0648,  0.    ,  0.1512,  0.1057,  0.1839,  0.195 ],
       [ 0.1484,  0.1081,  0.3182,  0.1597,  0.5148,  0.1778, -0

In [19]:
model.nlists

6

In [20]:
model.run_hockley_recog_single_sess()

# in model:
# print(self.att_vec[self.all_nos_unique[item_idx]-1])
# print(np.dot(self.c_old[:self.nitems_unique], self.f[:self.nitems_unique].T))

encoding
[0 5]
recognition
[0 5]
before item1 
 [[0.2828]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.2828]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.9165]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]]
c_old item1 
 [[0.5007]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.5007]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.7061]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]]
c_inp item2 
 [[0.7071]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.7071]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]]
encoding
[1 6]
recognition
[ 9 -1]
c_old item1 
 [[0.558 ]
 [0.2828]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.558 ]
 [0.2828]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.4286]
 [0.1833]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]]
c_inp item2 
 [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
encoding
[2 7]
recognition
[

In [21]:
model.M_FC

array([[0.5   , 0.3044, 0.2505, 0.2204, 0.1814, 0.5   , 0.3044, 0.2505, 0.2204, 0.1814, 0.1492, 0.1492, 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    ],
       [0.    , 0.5   , 0.127 , 0.1936, 0.1593, 0.    , 0.5   , 0.127 , 0.1936, 0.1593, 0.131 , 0.131 , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    ],
       [0.    , 0.    , 0.5   , 0.1496, 0.1231, 0.    , 0.    , 0.5   , 0.1496, 0.1231, 0.1012, 0.1012, 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    ],
       [0.    , 0.    , 0.    , 0.5   , 0.127 , 0.    , 0.    , 0.    , 0.5   , 0.127 , 0.234 , 0.234 , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.5   , 0.0601, 0.0601, 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    ],
       [0.5   , 0.3044, 0.2505, 0.2204, 0.1814, 0.5   , 0.3044, 0.2505, 0.2204, 0.1814, 0.1492, 0.1492, 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    ],
       [0.    , 0.5   , 0.127 , 0.

In [22]:
# model.att_vec

In [23]:
model.recog_similarity

[0.7081505060195923,
 0.0,
 0.569668173789978,
 0.0,
 0.864431619644165,
 0.9497739672660828]

In [24]:
recs = model.rec_items
rts = model.rec_times
csims = model.recog_similarity
result = np.column_stack((recs,rts,csims))
df_thin.loc[df_thin.session==sess, ['s_resp','s_rt','csim']] = result
df_simu = df_thin.merge(df_test,on=['session','itemno1'])
df_simu

,session,itemno1,itemno2_x,s_resp,s_rt,csim,itemno2_y
0,0,1,6,1.0,904.274231,0.708151,6
1,0,10,-1,0.0,1904.880859,0.000000,-1
2,0,3,7,1.0,1216.197876,0.569668,7
3,0,11,-1,0.0,1904.880859,0.000000,-1
4,0,4,9,1.0,647.223022,0.864432,9
